# Background Correction Notebook

In [ ]:
# /// script
# requires-python = ">=3.10"
# dependencies = [
#     "matplotlib",
#     "ndv[jupyter,vispy]",
#     "numpy",
#     "scikit-image",
#     "scipy",
#     "tifffile",
#     "imagecodecs",
# ]
# ///

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Overview</mark>

In this notebook, we will explore different approaches to **background correction** in fluorescence microscopy images. Background correction is a crucial pre-processing step that helps remove unwanted background signal and improves the quality of quantitative analysis. We will use the [**scikit-image**](https://scikit-image.org/docs/stable/) library to perform the background correction. Background subtraction is useful when the background is uniform and the signal to noise ratio is high.

We will demonstrate a simple background subtraction method using a sample fluorescence image. The main approaches we'll cover are:

- Subtracting a constant background value (e.g. mode or median of the image)
- Selecting and averaging background regions to determine background level

The choice of method depends on your specific imaging conditions and the nature of the background in your images. Here we'll demonstrate a basic approach that works well for images with relatively uniform background and distinct fluorescent signals.

<p class="alert alert-info">
    <strong>Note:</strong> Background correction should be done on raw images before any other processing steps. The corrected images can then be used for further analysis like segmentation and quantification.
</p>

The images we will use for this section can be downloaded from the <a href="../../../_static/data/07_measurement_and_quantification.zip" download> <i class="fas fa-download"></i> Measurement and Quantification Dataset</a>.

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Importing libraries</mark>

In [9]:
import matplotlib.pyplot as plt
import ndv
import numpy as np
import scipy
import skimage
import tifffile

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Background subtraction: mode subtraction </mark>

Background subtraction can be done in different ways. If the background dominates the image as in the example we will use, the most common pixel value (the **mode value**) can serve as a rough background estimate and we can subtract this value from the image.

Let's first load, display with `ndv` and explore the pixel values of the `07_bg_corr_nuclei.tif` image.

In [16]:
# raw image and labeled mask
image = tifffile.imread("../../_static/images/quant/07_bg_corr_nuclei.tif")

In [ ]:
ndv.imshow(image)

As you can notice, most of the pixel in the image belong to the background (anything but the nuclei). Therefore we can try to use the **mode value** of the image as a background estimate. We can use the `scipy.stats.mode` function to compute the mode of the image. 

Then, we can subtract the **mode value** from the image and print the minimum and maximum pixel values of the resulting image.

<p class="alert alert-info">
    <strong>Important:</strong> Before performing subtraction, convert the image to a <strong>floating-point</strong> (e.g., <code>image.astype(np.float32)</code>. This prevents <strong>unsigned integer underflow</strong>, which occurs when subtracting the mode value from pixels with intensities lower than the mode. In unsigned integer formats (like <code>uint16</code>), negative results wrap around to very large positive values (e.g., -1 becomes 65535), leading to incorrect results.
</p>

As you can see there are some negative values in the resulting image. This is because some pixels in the original image had values lower than the mode value, and when we subtract the mode from these pixels, we get negative values.

To keep working with the image, we need to handle these negative values. One common approach is to clip the negative values to zero, effectively setting any negative pixel values to zero. This is appropriate since we want background-corrected intensities to be greater than or equal to zero.

For that, we can use the `numpy` `np.clip` function to set any negative values to zero and then print the minimum and maximum pixel values of the resulting image.

Finally, we can visualize the background-corrected image (either with `matplotlib` or `ndv`):

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Background subtraction: selected regions</mark>

Sometimes the background isn't uniform, or the mode isn't representative. In these cases, we can manually choose one (or more) region we believe contains only background, estimate the average intensity in that region and subtract this average value from the image.

To select regions it might be helpful to first plot the images with the axis turned on, so we can estimate the pixel coordinates of the regions we want to select. Using `matplotlib`, we can even visualize and draw on the image the pixel values in the selected regions with the [`plt.gca().add_patch()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.add_patch.html#matplotlib-axes-axes-add-patch) function.

Now we can calculate the mean within the selected region.

As we did before, we can subtract this value from the image, and clip the result to ensure no negative values remain.

Finally, we can visualize the background-corrected image (either with `matplotlib` or `ndv`):

## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Background subtraction: rolling ball algorithm</mark>

Another way of performing background subtraction is the rolling ball algorithm. This is a method that uses a [rolling ball](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_rolling_ball.html) to estimate the background. It is a good method to use when the background is not uniform.

The radius parameter configures how distant pixels should be taken into account for determining the background intensity and should be a bit bigger than the size of the structures you want to keep.

Let's first load another image of a Drosophila embryo that has a non-uniform background:

In [71]:
# raw image
image = tifffile.imread("../../_static/images/quant/07_bg_corr_WF_drosophila.tif")

Let's explore the pixel values of the image with `ndv`:

In [ ]:
ndv.imshow(image)

We can now estimate the background by using the rolling ball algorithm. Since this function returns an image (which is the background we want to subtract), we can also visualize it with `matplotlib`:

We can now subtract the background residue from the image and plot with `matplotlib` the raw image, the background image, and the background-corrected image:


## <mark style="color: black; background-color: rgb(127,196,125); padding: 3px; border-radius: 5px;">Other Background Subtraction Techniques</mark>

These are more advanced or specialized techniques you can explore:

- **Morphological opening**: Removes small foreground objects to approximate the background.
- **Gaussian/median filtering**: Smooths out the image to isolate large-scale variations.
- **Polynomial surface fitting**: Useful when background varies gradually across the field.
- **Tiled/local background subtraction**: Estimate and subtract background patch-by-patch.

Your method choice should depend on image modality, signal-to-noise, and application.

A good reference for background correction is the [scikit-image documentation](https://scikit-image.org/docs/0.25.x/api/skimage.restoration.html).
